<a href="https://colab.research.google.com/github/boleamol/fake_news_classifier_using_LSTM/blob/main/Fake_news_classifier_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fake News Classifier Using Bidirectional LSTM 

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')   # connecting to my google drive

Mounted at /content/gdrive


In [4]:
!pwd

/content


In [6]:
#changing the working directory and entering to dataset directory
%cd drive/MyDrive/kaggle/fake_news/
!pwd

/content/drive/MyDrive/kaggle/fake_news
/content/drive/MyDrive/kaggle/fake_news


In [7]:
!ls

submit.csv  test.csv  train.csv


In [8]:
import tensorflow as tf

In [9]:
tf.__version__

'2.1.0'

In [10]:
import pandas as pd

In [11]:
df=pd.read_csv('train.csv')

In [12]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [13]:
###Drop Nan Values
df=df.dropna()

In [14]:
## Getting Independent Features in X
X=df.drop('label',axis=1)

In [15]:
## Get the Dependent features
y=df['label']

In [16]:
X.shape

(18285, 4)

In [17]:
y.shape

(18285,)

In [20]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [56]:
### Vocabulary size
voc_size=5000

In [57]:
messages=X.copy()

In [58]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [59]:
messages.reset_index(inplace=True)

In [60]:
import nltk
import re
from nltk.corpus import stopwords

In [61]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [62]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    #print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [63]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[3192, 1553, 278, 3530, 825, 2206, 2627, 115, 3933, 2009],
 [2372, 891, 967, 4435, 3654, 3719, 3296],
 [2675, 2672, 642, 4691],
 [2256, 2724, 821, 2888, 1606, 4806],
 [4336, 3654, 2557, 1932, 1016, 1477, 3654, 1303, 3506, 2963],
 [136,
  803,
  1479,
  4448,
  2822,
  1527,
  1015,
  1673,
  3409,
  4267,
  676,
  2672,
  4839,
  3528,
  3296],
 [1014, 1583, 1252, 4705, 1825, 4586, 1301, 2494, 2829, 4922, 2352],
 [2668, 276, 3267, 234, 3373, 917, 1527, 564, 2829, 4922, 2352],
 [1157, 4697, 687, 3317, 2299, 1544, 3076, 367, 1527, 581],
 [3321, 1370, 3024, 1499, 3575, 2333, 3901, 939],
 [4032, 4363, 1541, 1992, 3293, 1191, 1475, 3123, 203, 3060, 1693],
 [2888, 2373, 825, 1544, 1527, 3373],
 [1774, 3750, 3062, 3295, 1321, 1824, 4132, 2238, 4014],
 [224, 4304, 3414, 97, 3471, 1958, 2657, 2829, 4922, 2352],
 [4322, 4736, 3552, 1396, 2029, 2829, 4922, 2352],
 [3338, 3134, 4877, 844, 2263, 2336, 1320, 927, 4410, 634],
 [3946, 3639, 891],
 [4651, 3951, 685, 2540, 1527, 4042, 2356, 3296],
 [36

In [64]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  115 3933 2009]
 [   0    0    0 ... 3654 3719 3296]
 [   0    0    0 ... 2672  642 4691]
 ...
 [   0    0    0 ... 2829 4922 2352]
 [   0    0    0 ...  600 1108  681]
 [   0    0    0 ... 3013 1842  209]]


In [65]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3192,
       1553,  278, 3530,  825, 2206, 2627,  115, 3933, 2009], dtype=int32)

In [66]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [67]:
len(embedded_docs),y.shape

(18285, (18285,))

In [68]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [69]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [70]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)
# Dividing data in Train and Test

In [71]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=15,batch_size=64)

Train on 12250 samples, validate on 6035 samples
Epoch 1/15
12250/12250 [==============================] - 5s 381us/sample - loss: 0.3411 - accuracy: 0.8408 - val_loss: 0.1956 - val_accuracy: 0.9181
Epoch 2/15
12250/12250 [==============================] - 3s 224us/sample - loss: 0.1383 - accuracy: 0.9470 - val_loss: 0.1912 - val_accuracy: 0.9216
Epoch 3/15
12250/12250 [==============================] - 3s 221us/sample - loss: 0.0972 - accuracy: 0.9649 - val_loss: 0.2120 - val_accuracy: 0.9163
Epoch 4/15
12250/12250 [==============================] - 3s 224us/sample - loss: 0.0725 - accuracy: 0.9755 - val_loss: 0.2607 - val_accuracy: 0.9148
Epoch 5/15
12250/12250 [==============================] - 3s 219us/sample - loss: 0.0542 - accuracy: 0.9824 - val_loss: 0.2831 - val_accuracy: 0.9150
Epoch 6/15
12250/12250 [==============================] - 3s 223us/sample - loss: 0.0373 - accuracy: 0.9875 - val_loss: 0.3702 - val_accuracy: 0.9143
Epoch 7/15
12250/12250 [===========================

In [49]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [80]:
y_pred=model.predict_classes(X_test)
y_pred

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [0]], dtype=int32)

In [81]:
y_test

array([1, 0, 0, ..., 0, 1, 1])

In [82]:
from sklearn.metrics import confusion_matrix

In [83]:
confusion_matrix(y_test, y_pred)  # Printing confusion Matrix

array([[3141,  278],
       [ 259, 2357]])

In [84]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9110190555095278